<a href="https://colab.research.google.com/github/Ariq154404/IM-Project-2/blob/main/Jigsaw_FeatureEngg_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This part of the code was done in google cloud platform to do feature extraction.**

Here Tranformers are installed to take use of BERT from Huggingface

In [ ]:
!pip install transformers

Important library pertaining to the Feature processing are being installed

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

Here the train data is being read

In [ ]:
df_train=pd.read_csv('train.csv')

In [ ]:
df_train.head()


,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


Only column_Text and target are being chosen

In [ ]:
df_train=df_train[['comment_text','target']]

In [ ]:
df_train = df_train.sample(frac = 1)
df_train.reset_index(drop=True)

,comment_text,target
0,"Sorry China,we have a more important deal we n...",0.000000
1,I've dealt with this ER and Dr. on two differe...,0.000000
2,Your belief in a vengeful sky daddy doesn't tr...,0.166667
3,"Ah those Russians, have to give em credit for ...",0.400000
4,"""His handles were not elected , there is only ...",0.400000
...,...,...
1804869,"You know, it might go over well to refer to th...",0.455882
1804870,imagine. a lifestyle that lives in harmony wit...,0.700000
1804871,"Thank you, Paxima. I've been working here at m...",0.000000
1804872,"Mr. Dunlap's claims that ""WSF tries to make ou...",0.000000


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1804874 entries, 846242 to 579622
Data columns (total 2 columns):
 #   Column        Dtype  
---  ------        -----  
 0   comment_text  object 
 1   target        float64
dtypes: float64(1), object(1)
memory usage: 41.3+ MB


Only 90000 shuffeled rows are being chosed due to the complexity of the process

In [ ]:
df_train=df_train.head(90000)

In [ ]:
df_train.shape

(90000, 2)

In [ ]:
df_train.reset_index(inplace=True)

In [ ]:
df_train.to_csv('small_train.csv')

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90000 entries, 0 to 89999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         90000 non-null  int64  
 1   comment_text  90000 non-null  object 
 2   target        90000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.1+ MB


In [ ]:
df_train['target'].describe()

count    90000.000000
mean         0.102637
std          0.196530
min          0.000000
25%          0.000000
50%          0.000000
75%          0.166667
max          1.000000
Name: target, dtype: float64

In [ ]:
df_test=pd.read_csv('test.csv')

In [ ]:
df_test.head()

,id,comment_text
0,7097320,[ Integrity means that you pay your debts.]\n\...
1,7097321,This is malfeasance by the Administrator and t...
2,7097322,@Rmiller101 - Spoken like a true elitist. But ...
3,7097323,"Paul: Thank you for your kind words. I do, in..."
4,7097324,Sorry you missed high school. Eisenhower sent ...


In [ ]:
df_test.shape

(97320, 2)

In [ ]:
df_train.head()

,index,comment_text,target
0,846242,"Sorry China,we have a more important deal we n...",0.000000
1,896179,I've dealt with this ER and Dr. on two differe...,0.000000
2,1305007,Your belief in a vengeful sky daddy doesn't tr...,0.166667
3,1153675,"Ah those Russians, have to give em credit for ...",0.400000
4,934343,"""His handles were not elected , there is only ...",0.400000


In [ ]:
df_train.shape

(90000, 3)

Total comment_text from test and train are being appended 

In [ ]:
tot_comment=pd.concat((df_train['comment_text'],df_test['comment_text']),axis=0)

In [ ]:
tot_comment.head()

0    Sorry China,we have a more important deal we n...
1    I've dealt with this ER and Dr. on two differe...
2    Your belief in a vengeful sky daddy doesn't tr...
3    Ah those Russians, have to give em credit for ...
4    "His handles were not elected , there is only ...
Name: comment_text, dtype: object

In [ ]:
tot_comment.shape

(187320,)

**Here the pretrained weights are being downloaded along with the DistilBertTokenizer which is a light version of the transformer**

In [ ]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
tot_comment.shape

(187320,)

In [ ]:
tot_comment[:5]

0    Sorry China,we have a more important deal we n...
1    I've dealt with this ER and Dr. on two differe...
2    Your belief in a vengeful sky daddy doesn't tr...
3    Ah those Russians, have to give em credit for ...
4    "His handles were not elected , there is only ...
Name: comment_text, dtype: object

The text column are being tokeninzed to be later fed on to the model

In [ ]:
tokenized = tot_comment[:].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,truncation=True,max_length=512)))

In [ ]:
tokenized.shape

(187320,)

Finally the tokenized values are saved

In [ ]:
from numpy import save
save('tokenized.npy', tokenized)